In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,150.000000,150.000000
mean,0.184803,0.523041
std,0.150672,2.069191
min,0.000000,0.007332
25%,0.000000,0.100060
50%,0.190909,0.146972
75%,0.282468,0.284729
max,0.750000,18.006205


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1534
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.696
Time:                        20:03:10   Log-Likelihood:                -321.34
No. Observations:                 150   AIC:                             646.7
Df Residuals:                     148   BIC:                             652.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6047      0.269      2.251      0.026       0.074       1.136
X             -0.4419      1.128     -0.392      0.696      -2.672       1.788
==============================================================================
Omnibus:                      257.159   Durbin-Watson:                   0.877
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27283.431
Skew:                           7.980   Prob(JB):                         0.00
Kurtosis:                      67.114   Cond. No.                         6.89
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1818
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.670
Time:                        20:03:10   Log-Likelihood:                -310.43
No. Observations:                 150   AIC:                             624.9
Df Residuals:                     148   BIC:                             630.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7127      0.250      2.853      0.005       0.219       1.206
X              0.4474      1.049      0.426      0.670      -1.626       2.521
==============================================================================
Omnibus:                      263.320   Durbin-Watson:                   0.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30652.787
Skew:                           8.312   Prob(JB):                         0.00
Kurtosis:                      71.030   Cond. No.                         6.89
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,132.000000,132.000000
mean,0.194725,0.167597
std,0.142258,0.108434
min,0.000000,0.007332
25%,0.090909,0.092965
50%,0.200000,0.133408
75%,0.285714,0.205683
max,0.545455,0.458297


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     2.950
Date:                Thu, 11 May 2023   Prob (F-statistic):             0.0883
Time:                        20:03:11   Log-Likelihood:                 107.94
No. Observations:                 132   AIC:                            -211.9
Df Residuals:                     130   BIC:                            -206.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1455      0.016      9.138      0.000       0.114       0.177
X              0.1135      0.066      1.717      0.088      -0.017       0.244
==============================================================================
Omnibus:                       21.927   Durbin-Watson:                   0.826
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.642
Skew:                           1.110   Prob(JB):                     9.94e-07
Kurtosis:                       3.314   Cond. No.                         7.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.182
Date:                Thu, 11 May 2023   Prob (F-statistic):             0.0768
Time:                        20:03:11   Log-Likelihood:                 170.25
No. Observations:                 132   AIC:                            -336.5
Df Residuals:                     130   BIC:                            -330.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0685      0.010      6.901      0.000       0.049       0.088
X              0.0735      0.041      1.784      0.077      -0.008       0.155
==============================================================================
Omnibus:                       26.326   Durbin-Watson:                   1.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.463
Skew:                           1.186   Prob(JB):                     3.28e-08
Kurtosis:                       3.803   Cond. No.                         7.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""